In [1]:
import numpy as np
import time
import math
from numpy.linalg import qr
from sklearn.metrics import mean_squared_error
from scipy.linalg.interpolative import estimate_spectral_norm
from ipynb.fs.full.Traditional_PCA import do_pca
from ipynb.fs.full.SyntheticDataGeneration import generate_gaussian_dataset_concept_drift

# DBPCA Algorithm
Chun Liang Li, Hsuan Ten Lin, and Chi Jen Lu. 2016. Rivalry of two families of algorithms for memory-restricted streaming PCA. In Proceedings of the 19th International Conference on Artificial Intelligence and Statistics, AISTATS 2016.

In [2]:
def dbpca(X, B, k, d):
    """
    B : int
        block size or window size
    X : numpy array d x n 
        n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    k : int
        number of principal components
    """
    S0 = None
    for i in range(k):
        normal1 = np.random.normal(0, 1, d).reshape((-1,1))
        if S0 is None:
            S0 = normal1
        else:
            S0 = np.hstack((S0,normal1))
    Q, R = np.linalg.qr(S0)
    index = 0
    for i in range(int(n/B)):
        # initialize matrix S of size dxk
        S = np.zeros((d, k))
        for j in range(B):
            x = X[:,index].reshape((-1,1))
            S = S + (1/B) * x.dot(x.T).dot(Q)
            index += 1
        Q, R = np.linalg.qr(S)
    return Q

# Modified DBPCA

In [3]:
def update_Q(W,d,k,Q):
    """
    Update estimated eigen vector for each window
    Parameters
    ----------
    W : a matrix, numpy array of d x m 
        window or buffer consisting of m data points of d attributes
    d : int
        number of attributes
    k : int
        number of principle component
    Q : a matrix, numpy array of d x k
        estimated eigenvectors from previous window
    Returns
    -------
    Q : a matrix, numpy array of d x k
        updated estimated eigenvectors
    """
    S = np.zeros((d,k))
    B = W.shape[1]
    for j in range (B):
        x = W[:,j].reshape((-1,1))
        S = S + (1/B) * x.dot(x.T).dot(Q)
    Q, R = np.linalg.qr(S)
    return Q

def initialize_Q(d,k):
    """
    Initialize estimated eigen vectors 
    Parameters
    ----------
    d : int
        number of attributes
    k : int
        number of principle component
    Returns
    -------
    Q : a matrix, numpy array of d x k
        initial estimated eigenvectors
    """
    S0 = None
    for i in range(k):
        normal1 = np.random.normal(0, 1, d).reshape((-1,1))
        if S0 is None:
            S0 = normal1
        else:
            S0 = np.hstack((S0,normal1))
    Q, R = np.linalg.qr(S0)
    return Q
        
def dbpca_with_forgetting_factor(B, X, k, d, f):
    """
    B : int
        block size or window size
    X : numpy array d x n 
        n is the number of data points and d is the number of attributes
    output q = estimated eigen vectors of cov(X)
    k : int
        number of principal components
    f : float
        forgetting factor (0,1]
    """
    n = X.shape[1]
    Q = initialize_Q(d, k)
    for i in range(int(n/B)):
        S = np.zeros((d, k))
        start = i*B
        end = (i*B)+B
        W = X[:,start:end]
        Q = Q * math.pow(f,2)
        Q = update_Q(W,d,k,Q)
    return Q